In [6]:
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
import os

In [7]:
# Preprocess audio data
def preprocess_audio(file_path, target_duration=10):
    audio, _ = librosa.load(file_path, sr=None)

    target_length = int(target_duration * _)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]

    mfccs = librosa.feature.mfcc(y=audio, sr=_, n_mfcc=13)
    return mfccs

In [8]:
# Load the first two models from the 'final_model' directory and delete the rest
def load_and_cleanup_models():
    final_model_directory = 'final_model'

    # List all files in the directory
    all_files = os.listdir(final_model_directory)

    # Filter only '.h5' files
    h5_files = [file for file in all_files if file.endswith('.h5')]

    # Sort files based on modification time
    h5_files.sort(key=lambda x: os.path.getmtime(os.path.join(final_model_directory, x)))

    # Keep the first two models and delete the rest
    models_to_keep = h5_files[:2]

    for file in h5_files[2:]:
        file_path = os.path.join(final_model_directory, file)
        os.remove(file_path)
        print(f"Deleted model: {file_path}")

    # Load the first model
    if models_to_keep:
        first_model_path = os.path.join(final_model_directory, models_to_keep[0])
        model1 = load_model(first_model_path)
        print(f"Loaded first model: {first_model_path}")
    else:
        model1 = None

    return model1


In [9]:
# Load the trained model
model = load_and_cleanup_models()

# Example: Make predictions on unseen data
unseen_data_path = 'unseen_sample.wav'
# unseen_data_path = 'hindi_hate_speech_model.h5'
unseen_audio = preprocess_audio(unseen_data_path)

# Ensure that the input shape matches the model's expected input shape
unseen_audio = unseen_audio[np.newaxis, ..., np.newaxis]

# Make predictions
predictions = model.predict(unseen_audio)

# Threshold the predictions for binary classification
binary_predictions = (predictions > 0.5).astype(int)

# Print the predictions
# If binary_predictions is [1], it means the model predicts the input audio contains hate speech. 
# If it is [0], it means the model predicts the input audio does not contain hate speech.

print("Predictions:", binary_predictions.flatten())

Loaded first model: final_model/final_model_20240117_092228.h5
1/1 [==============================] - 0s 239ms/step
Predictions: [0]


In [10]:
# Print the raw probability scores
print("Raw Probabilities:", predictions.flatten())

Raw Probabilities: [0.23563124]
